In [9]:
import sys
import os
sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath(''))

import pandas as pd
import numpy as np
import jsonlines
import seaborn as sns
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from facebook_hateful_memes_detector.utils import read_json_lines_into_df, clean_text


In [2]:
dev = read_json_lines_into_df('../data/dev.jsonl')
train = read_json_lines_into_df('../data/train.jsonl')
test = read_json_lines_into_df('../data/test.jsonl')

In [3]:
dev.shape
train.shape
test.shape

(500, 4)

(8500, 4)

(1000, 3)

In [4]:
train.label.value_counts()
dev.label.value_counts()

0    5450
1    3050
Name: label, dtype: int64

1    250
0    250
Name: label, dtype: int64

In [5]:
columns = train.columns
print(columns)

Index(['id', 'img', 'label', 'text'], dtype='object')


In [6]:
train.shape

(8500, 4)

In [ ]:
import fasttext
tokenised_lengths = train.text.apply(clean_text).apply(fasttext.tokenize).apply(len)
tokenised_lengths[tokenised_lengths > 64]
sum(tokenised_lengths > 64)
tokenised_lengths.describe()
train.text[tokenised_lengths > 64].values

In [8]:
from transformers import AlbertModel, AlbertTokenizer
import torch
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
tokenised_lengths = train.text.apply(lambda x: tokenizer.encode(x, add_special_tokens=True)).apply(len)
tokenised_lengths.describe()

count    8500.000000
mean       16.795059
std         9.058886
min         3.000000
25%        10.000000
50%        15.000000
75%        21.000000
max        87.000000
Name: text, dtype: float64

In [ ]:
texts = list(train.text.values)
type(texts)
texts

In [35]:
def mapper(texts):
    m = lambda x: tokenizer.encode_plus(x, add_special_tokens=True, pad_to_max_length=True, max_length=128)
    input_ids, attention_mask = zip(*[(d['input_ids'], d['attention_mask']) for d in map(m, texts)])
    return torch.tensor(input_ids), torch.tensor(attention_mask)
input_ids, attention_mask = mapper(texts[:64])
input_ids.shape
attention_mask.shape

torch.Size([64, 128])

torch.Size([64, 128])

In [36]:
model = AlbertModel.from_pretrained('albert-base-v2')

In [38]:
outputs = model(input_ids, attention_mask=attention_mask)
outputs[0].shape
outputs[1].shape

torch.Size([64, 128, 768])

torch.Size([64, 768])

In [50]:
import torch.nn as nn
import torch
conv1d = nn.Conv1d(128, 16, 1, stride=1, padding=0, dilation=1, groups=8, bias=False, padding_mode='zeros')
conv1d(outputs[0]).shape


torch.Size([64, 16, 768])

In [ ]:
from PIL import Image
input_image = Image.open("../data/img/01235.png")
input_image.copy()